In [ ]:
from kafka import KafkaConsumer
import pandas as pd

# To consume latest messages and auto-commit offsets
consumer = KafkaConsumer('airpollution_from_API', bootstrap_servers=['localhost:9092'], auto_offset_reset='latest')
consumer2 = KafkaConsumer('weather_from_API', bootstrap_servers=['localhost:9093'], auto_offset_reset='latest')
consumer3 = KafkaConsumer('airpollution_from_sensor', bootstrap_servers=['localhost:9094'], auto_offset_reset='latest')
from datetime import datetime
import time
# subscribe to the topic and save the data to a csv file
curr_time = datetime.now()
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'])
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS airpollution 
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")
session.set_keyspace('airpollution')

# Create Tables
session.execute("""
    CREATE TABLE IF NOT EXISTS airpollution_data (
        district text,
        timestamp bigint,
        lon float,
        lat float,
        AQI int,
        CO float,
        NO float,
        NO2 float,
        O3 float,
        SO2 float,
        PM2_5 float,
        PM10 float,
        NH3 float,
        PRIMARY KEY (district, timestamp)
    )
""")
while True:

    for message in consumer:
        print(f"Message received: {message.value}")
        # Format the data to be a dataframe
        data = {}
        import json
        message_value = json.loads(message.value.decode('utf-8'))
        data['district'] = message_value['district']
        data['timestamp'] = message_value['timestamp']
        data['lon'] = message_value['lon']
        data['lat'] = message_value['lat']
        data['AQI'] = message_value['AQI']
        data['CO'] = message_value['CO']
        data['NO'] = message_value['NO']
        data['NO2'] = message_value['NO2']
        data['O3'] = message_value['O3']
        data['SO2'] = message_value['SO2']
        data['PM2_5'] = message_value['PM2_5']
        data['PM10'] = message_value['PM10']
        data['NH3'] = message_value['NH3']
        data = pd.DataFrame(data, index=[0])
        print(f"Data to insert: {data}")
        data.to_csv('airpollution_from_API.csv', mode='a', header=False, index=False)
        print("Inserting data into Cassandra...")
        session.execute("""
            INSERT INTO airpollution_data (district, timestamp, lon, lat, AQI, CO, NO, NO2, O3, SO2, PM2_5, PM10, NH3)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (data['district'].iloc[0], int(data['timestamp'].iloc[0]), float(data['lon'].iloc[0]), float(data['lat'].iloc[0]), int(data['AQI'].iloc[0]), float(data['CO'].iloc[0]), float(data['NO'].iloc[0]), float(data['NO2'].iloc[0]), float(data['O3'].iloc[0]), float(data['SO2'].iloc[0]), float(data['PM2_5'].iloc[0]), float(data['PM10'].iloc[0]), float(data['NH3'].iloc[0]))
        ) # iloc[0] is used to get the first element of the dataframe
        
    
